# Determining Equivalance Relative Spectral Response

In [ ]:
import numpy as np
import math
from scipy.interpolate import UnivariateSpline
from scipy import signal
from scipy import ndimage
from scipy import stats
from scipy.stats import norm
import emd # From - https://github.com/andreasjansson/python-emd
import pandas
import plotly

plotly.offline.init_notebook_mode()

In [ ]:
# Plot every sensor band for a given sensor
def plotallbands_comparative(sensors):
    plotlydatalist = []
    for sensor, color in sensors:
        
        for key in sensordict[sensor].keys():
            plotlydatalist.append({"x": sensordict[sensor][(key)].wavelength,\
                                    "y": sensordict[sensor][(key)].rsr,"name": (sensor+' '+key), "line": dict(color = (color), width = 1)})

    plotly.offline.iplot({ "data": plotlydatalist,"layout": {"title": str(sensors)}})

In [ ]:
# Plot every sensor band for a given sensor
def plotallbands(sensor):
    plotlydatalist = []
    for key in sensordict[sensor].keys():
        plotlydatalist.append({"x": sensordict[sensor][(key)].wavelength,\
                                "y": sensordict[sensor][(key)].rsr,"name": (str(key))})

    plotly.offline.iplot({ "data": plotlydatalist,"layout": {"title": sensor}})

## Import data and configure for common access

In [ ]:
spot1 = pandas.read_excel('r453_9_spectralsensivity.xls', sheet_name = 'SPOT 1', header=4, skiprows=[42,43]) #hrv1, hrv2

spot2 = pandas.read_excel('r453_9_spectralsensivity.xls', sheet_name = 'SPOT 2', header=4, skiprows=[42,43]) #hrv1, hrv2

spot3 = pandas.read_excel('r453_9_spectralsensivity.xls', sheet_name = 'SPOT 3', header=4, skiprows=[42,43]) #hrv1, hrv2

spot4 = pandas.read_excel('r453_9_spectralsensivity.xls', sheet_name = 'SPOT 4', header=4, skiprows=[68,69]) #hrvir1, hrvir2

spot5 = pandas.read_excel('r453_9_spectralsensivity.xls', sheet_name = 'SPOT 5', header=6) #hrg1, hrg2




qb02 = pandas.read_excel('QB02_WV01_WV02_Spectral_Response_Final_Correct_Jan292013.xlsx',\
                               sheet_name = 'QB02 Relative Radiance Response', header=7)
wv01 = pandas.read_excel('QB02_WV01_WV02_Spectral_Response_Final_Correct_Jan292013.xlsx',\
                               sheet_name = 'WV01 Relative Radiance Response', header=7)
geoeye1 = pandas.read_excel('GeoEye-1_Relative_Spectral_Response.xls', sheet_name = 'Sheet1', header = 26)
geoeye1 = geoeye1.dropna()
ls8oli = pandas.read_excel('Ball_BA_RSR.xlsx',sheet_name = None)
ls8tirs = pandas.read_excel('TIRS_Relative_Spectral_ResponsesBA.xlsx', sheet_name = ['TIRS BA RSR'])
ls5tm = pandas.read_excel('L5_TM_RSR.xlsx',sheet_name = None)
ls7etm = pandas.read_excel('L7_RSR.xlsx',sheet_name = ['Blue-L7','Green-L7','Red-L7','NIR-L7', 'SWIR(5)-L7','SWIR(7)-L7','Pan-L7'])
# Band8 has some dodgy outliers i.e. bimodal with zeros between - two leading values removed in spreadsheet
#s2amsi = pandas.read_excel('Sentinel-2A MSI Spectral Responses.xlsx',sheet_name = ['Spectral Responses'])
s2amsi = pandas.read_excel('Sentinel-2 MSI Spectral Responses.xlsx',sheet_name = ['Spectral Responses (S2A)'])
s2bmsi = pandas.read_excel('Sentinel-2 MSI Spectral Responses.xlsx',sheet_name = ['Spectral Responses (S2B)'])
tmodis = pandas.read_excel('MODIS_FM1_IB_OOB_RSR_merged.xls', sheet_name = ['modis_fm1_rsp_mid_chnl_sum'], header=0)
tmodis['modis_fm1_rsp_mid_chnl_sum'] = tmodis['modis_fm1_rsp_mid_chnl_sum'].dropna()
#ASTER - http://asterweb.jpl.nasa.gov/content/01_mission/03_instrument/archive/swir.txt (vnir.txt, tir.txt)
astervnir = pandas.read_csv('aster/vnir.txt',delim_whitespace=True, header=3)
asterswir = pandas.read_csv('aster/swir.txt',delim_whitespace=True, header=3)
astertir = pandas.read_csv('aster/tir.txt',delim_whitespace=True, header=3)
#H8 - http://www.data.jma.go.jp/mscweb/en/himawari89/space_segment/spsg_ahi.html
viirs = pandas.read_excel('VIIRS.xlsx', sheet_name = ['I1', 'I2', 'I3', 'I4', 'I5', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16A', 'M16B'])
h8ahi = pandas.read_excel('h8/AHI-8_SpectralResponsivity_Data.xlsx', sheet_name = ['Band 1','Band 2','Band 3','Band 4','Band 5','Band 6','Band 7','Band 8','Band 9','Band 10','Band 11','Band 12','Band 13','Band 14','Band 15','Band 16'], header = [4])
irradiance = pandas.read_excel('solar_irradiance/ASTMG173.xls', sheet_name = ['SMARTS2'], header=1)
print ("MODIS:",type(tmodis),"ASTER:",type(astervnir), "Irradiance:", type(irradiance))


In [ ]:
print ("SPOT 1", spot1.keys())
print ("SPOT 2", spot2.keys())
print ("SPOT 3", spot3.keys())
print ("SPOT 4", spot4.keys())
print ("SPOT 5", spot5.keys())
print ("Geoeye 1", geoeye1.keys())
print ("Worldview 1", wv01.keys())
print ("Quickbird 2", qb02.keys())
print ("ASTER VNIR", astervnir.keys())
print ("ASTER TIR", astertir.keys())
print ("ASTER SWIR", asterswir.keys())
print ("VIIRS", viirs.keys())
print ("Himawari 8 AHI", h8ahi.keys())
print ("OLI ",ls8oli.keys())
print ("TIRS", ls8tirs.keys())
print ("TM", ls5tm.keys())
print ("ETM+", ls7etm.keys())
print ("S2AMSI", s2amsi.keys())
print ("S2BMSI", s2bmsi.keys())
print ("Terra MODIS", (tmodis.keys()))
print ("Solar Irradiance", (irradiance.keys()))
sensorlist = [spot1, spot2, spot3, spot4, spot5, geoeye1, wv01, qb02, astervnir,asterswir,astertir, h8ahi,ls8oli,ls8tirs,ls5tm,ls7etm,s2amsi,tmodis]

In [ ]:
def updatecolumns(df):
    for key in df.keys():
        if len(df[key].columns) == 3:
            df[key].columns = ['Wavelength', 'RSR', 'std']   
        if len(df[key].columns) == 2:
            df[key].columns = ['Wavelength', 'RSR']
    return(df)

ls8oli = updatecolumns(ls8oli)

for key in h8ahi.keys():
    h8ahi[key].columns = ['Wavelength', 'Wavenumber', 'RSR']
for key in viirs.keys():    
    viirs[key].columns = ['Wavelength', 'RSR']
spot5.columns = ['Wavelength', 'Pa','B1','B2','B3','MIR','Pa.1','B1.1','B2.1','B3.1','MIR.1']   
geoeye1.columns = ['Wavelength', 'Panchromatic', 'Blue', 'Green', 'Red','Near IR']
wv01.columns = ['Wavelength', 'Pan']
qb02.columns = ['Wavelength', 'Pan', 'Blue', 'Green', 'Red','NIR']
astervnir.columns = ['Wavelength1','VNIR_Band1','Wavelength2','VNIR_Band2','Wavelength3','VNIR_Band3', 'Wavelength3B','VNIR_Band3B']
asterswir.columns = ['Wavelength4','SWIR_Band4','Wavelength5','SWIR_Band5','Wavelength6','SWIR_Band6','Wavelength7','SWIR_Band7','Wavelength8','SWIR_Band8','Wavelength9','SWIR_Band9']
astertir.columns = ['Wavelength10','TIR_Band10','Wavelength11','TIR_Band11','Wavelength12','TIR_Band12','Wavelength13','TIR_Band13','Wavelength14','TIR_Band14']
ls8tirs['TIRS BA RSR'].columns = ['Wavelength', 'TIRS1RSR', 'TIRS2RSR', 'ignore','ignore','ignore','ignore','ignore']
ls7etm = updatecolumns(ls7etm)
ls5tm = updatecolumns(ls5tm)
s2amsi['Spectral Responses (S2A)'].columns = ['Wavelength', 'Band1', 'Band2', 'Band3', 'Band4', 'Band5',
       'Band6', 'Band7', 'Band8', 'Band8A', 'Band9', 'Band10',
       'Band11', 'Band12']
s2bmsi['Spectral Responses (S2B)'].columns = ['Wavelength', 'Band1', 'Band2', 'Band3', 'Band4', 'Band5',
       'Band6', 'Band7', 'Band8', 'Band8A', 'Band9', 'Band10',
       'Band11', 'Band12']
tmodis['modis_fm1_rsp_mid_chnl_sum'].columns = ['Wavelength1', 'Band1RSR', 'Wavelength2', 'Band2RSR', 'Wavelength3',
       'Band3RSR', 'Wavelength4', 'Band4RSR', 'Wavelength5', 'Band5RSR',
       'Wavelength6', 'Band6RSR', 'Wavelength7', 'Band7RSR', 'Wavelength8',
       'Band8RSR', 'Wavelength9', 'Band9RSR', 'Wavelength10', 'Band10RSR',
       'Wavelength11', 'Band11RSR', 'Wavelength12', 'Band12RSR', 'Wavelength13',
       'Band13RSR', 'Wavelength14', 'Band14RSR', 'Wavelength15', 'Band15RSR',
       'Wavelength16', 'Band16RSR', 'Wavelength17', 'Band17RSR', 'Wavelength18',
       'Band18RSR', 'Wavelength19', 'Band19RSR', 'Wavelength20', 'Band20RSR',
       'Wavelength21', 'Band21RSR', 'Wavelength22', 'Band22RSR', 'Wavelength23',
       'Band23RSR', 'Wavelength24', 'Band24RSR', 'Wavelength25', 'Band25RSR',
       'Wavelength26', 'Band26RSR', 'Wavelength27', 'Band27RSR', 'Wavelength28',
       'Band28RSR', 'Wavelength29', 'Band29RSR', 'Wavelength30', 'Band30RSR',
       'Wavelength31', 'Band31RSR', 'Wavelength32', 'Band32RSR', 'Wavelength33',
       'Band33RSR', 'Wavelength34', 'Band34RSR', 'Wavelength35', 'Band35RSR',
       'Wavelength36', 'Band36RSR']
irradiance['SMARTS2'].columns = ['Wavelength', 'ETR', 'Global_Tilt', 'Direct_Circumsolar']


In [ ]:
for i in spot5.keys():
    if i == 'Wavelength':
        print('skipping Wavelength')
    else:
        spot5[i] = spot5[i].fillna(0)
spot5 = spot5.dropna()    

In [ ]:
# Combine as commonly references dict of dataframes combinedsensors['sensor_band'].wavelength
# Define a sensible dataframe to hold the sensor info - above is getting blurghy
# Room to add sensor native sampling resolution too though we can consider that out of scope for now

# combinedsensors = pandas.DataFrame([{ "wavelength": , "rsr":  }], dtype = 'sensor')

# A dict of pandas frames sounds like a reasonable container for this

sensordict = {}

#TODO def modes/types

# Type 1 - dict of bands i.e. one band per sheet, one wavelength column, one rsr columns

sensor = 'ls8oli'
sensordict[sensor] = {}
inputdict = ls8oli

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors

sensor = 'ls5tm'
sensordict[sensor] = {}
inputdict = ls5tm

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors

sensor = 'ls7etm'
sensordict[sensor] = {}
inputdict = ls7etm

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength)
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors
    
sensor = 'h8ahi'
sensordict[sensor] = {}
inputdict = h8ahi

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*1000 )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors   

sensor = 'viirs'
sensordict[sensor] = {}
inputdict = viirs    

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors      
 
# Type 2 - single dict i.e. multiple bands per sheet, one wavelength column, multiple rsr columns 
    

def multiplebandsonewavelength(inputdict, sensor, wvscale):
    sensordict[sensor] = {}
    for key in inputdict.keys():

        if (type(inputdict) is dict) or (str(type(inputdict)) == "<class 'collections.OrderedDict'>"):
            for column in inputdict[(key)].columns:
                
                if not (column == 'Wavelength'):

                    sensorrsr = pandas.Series(inputdict[(key)][column])
                    
                    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*wvscale )
                    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})    
                    sensordict[sensor][(column)] = combinedsensors
        else:

            for column in inputdict.columns:
                if not (column == 'Wavelength'):

                    sensorrsr = pandas.Series(inputdict[column])

                    sensorwavelength = pandas.Series(inputdict.Wavelength*wvscale )
                    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})

                    sensordict[sensor][(column)] = combinedsensors

multiplebandsonewavelength(s2amsi, 's2amsi',1)
multiplebandsonewavelength(s2bmsi, 's2bmsi',1)
multiplebandsonewavelength(geoeye1, 'geoeye1',1)
multiplebandsonewavelength(wv01, 'wv01',1000)
multiplebandsonewavelength(qb02,'qb02',1000)
multiplebandsonewavelength(spot5, 'spot5',1)

In [ ]:
# SPOT needs reorganisation

def spotdataframe(sensor,inputframe):
    sensordict[sensor] = {}
    inputdict = inputframe
    for column in inputdict.columns:
        if not (column == 'Spectral band'):
            start = inputdict[column][0]
            step = inputdict[column][1]
            offset=2
            numsteps = inputdict[column].count()-offset
            wavelength = np.linspace(start,start+((numsteps-1)*step),numsteps)
            rsr = inputdict[column][2:len(wavelength)+2]
            sensordict[sensor][column] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})
spotdataframe('spot1',spot1)
spotdataframe('spot2',spot2)
spotdataframe('spot3',spot3)
spotdataframe('spot4',spot4)
plotallbands('spot4')

In [ ]:
sensor = 'ls8tirs'
sensordict[sensor] = {}
inputdict = ls8tirs
for key in inputdict.keys():
    for column in inputdict[(key)].columns:
        if not (column == 'Wavelength') and not (column == 'ignore'):
            sensorrsr = pandas.Series(inputdict[(key)][column])
            sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
            combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
            
            sensordict[sensor][(column)] = combinedsensors            

In [ ]:
# Type 3 - one band in dict i.e. alternating wavelength and column  per sheet, one wavelength column, one rsr columns
# Combine all of the ASTER telescope to a single dictionary - leaving OLI and TIRS seperate for now

sensor = 'aster'
sensordict[sensor] = {}
inputdict = astervnir
sensordict[sensor][('VNIR_Band1')] = pandas.DataFrame({'wavelength': astervnir.Wavelength1*1000, 'rsr': astervnir.VNIR_Band1})
sensordict[sensor][('VNIR_Band2')] = pandas.DataFrame({'wavelength': astervnir.Wavelength2*1000, 'rsr': astervnir.VNIR_Band2})
sensordict[sensor][('VNIR_Band3')] = pandas.DataFrame({'wavelength': astervnir.Wavelength3*1000, 'rsr': astervnir.VNIR_Band3})
sensordict[sensor][('VNIR_Band3B')] = pandas.DataFrame({'wavelength': astervnir.Wavelength3B*1000, 'rsr': astervnir.VNIR_Band3B})
sensordict[sensor][('SWIR_Band4')] = pandas.DataFrame({'wavelength': asterswir.Wavelength4*1000, 'rsr': asterswir.SWIR_Band4})
sensordict[sensor][('SWIR_Band5')] = pandas.DataFrame({'wavelength': asterswir.Wavelength5*1000, 'rsr': asterswir.SWIR_Band5})
sensordict[sensor][('SWIR_Band6')] = pandas.DataFrame({'wavelength': asterswir.Wavelength6*1000, 'rsr': asterswir.SWIR_Band6})
sensordict[sensor][('SWIR_Band7')] = pandas.DataFrame({'wavelength': asterswir.Wavelength7*1000, 'rsr': asterswir.SWIR_Band7})
sensordict[sensor][('SWIR_Band8')] = pandas.DataFrame({'wavelength': asterswir.Wavelength8*1000, 'rsr': asterswir.SWIR_Band8})
sensordict[sensor][('SWIR_Band9')] = pandas.DataFrame({'wavelength': asterswir.Wavelength9*1000, 'rsr': asterswir.SWIR_Band9})
sensordict[sensor][('TIR_Band10')] = pandas.DataFrame({'wavelength': astertir.Wavelength10*1000, 'rsr': astertir.TIR_Band10})
sensordict[sensor][('TIR_Band11')] = pandas.DataFrame({'wavelength': astertir.Wavelength11*1000, 'rsr': astertir.TIR_Band11})
sensordict[sensor][('TIR_Band12')] = pandas.DataFrame({'wavelength': astertir.Wavelength12*1000, 'rsr': astertir.TIR_Band12})
sensordict[sensor][('TIR_Band13')] = pandas.DataFrame({'wavelength': astertir.Wavelength13*1000, 'rsr': astertir.TIR_Band13})
sensordict[sensor][('TIR_Band14')] = pandas.DataFrame({'wavelength': astertir.Wavelength14*1000, 'rsr': astertir.TIR_Band14})


In [ ]:
# Type 4 - Solar Irradiance - supposing we normalise this or find a normalised source other than 
# http://www.astm.org/Standards/G173.htm 
sensor = 'irradiance'
sensordict[sensor] = {}
inputdict = irradiance
sensordict[sensor][('solar')] = pandas.DataFrame({'wavelength': \
                                                       irradiance['SMARTS2'].Wavelength,\
                                                       'rsr': \
                                                       irradiance['SMARTS2'].Direct_Circumsolar})
solar_curve = ndimage.filters.gaussian_filter(sensordict['irradiance'][('solar')].rsr, 20)


In [ ]:
# Similar to above - could build an iterator  - some more on this below

sensor = 'tmodis'
sensordict[sensor] = {}
inputdict = tmodis
tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength4*1000
tmodis['modis_fm1_rsp_mid_chnl_sum'].Band4RSR

sensordict[sensor][('Band1')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength1*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band1RSR})
sensordict[sensor][('Band2')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength2*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band2RSR})
sensordict[sensor][('Band3')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength3*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band3RSR})
sensordict[sensor][('Band4')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength4*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band4RSR})
sensordict[sensor][('Band5')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength5*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band5RSR})
sensordict[sensor][('Band6')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength6*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band6RSR})
sensordict[sensor][('Band7')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength7*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band7RSR})
sensordict[sensor][('Band8')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength8*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band8RSR})
sensordict[sensor][('Band9')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength9*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band9RSR})
sensordict[sensor][('Band10')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength10*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band10RSR})
sensordict[sensor][('Band11')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength11*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band11RSR})
sensordict[sensor][('Band12')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength12*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band12RSR})
sensordict[sensor][('Band13')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength13*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band13RSR})
sensordict[sensor][('Band14')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength14*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band14RSR})
sensordict[sensor][('Band15')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength15*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band15RSR})
sensordict[sensor][('Band16')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength16*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band16RSR})
sensordict[sensor][('Band17')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength17*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band17RSR})
sensordict[sensor][('Band18')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength18*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band18RSR})
sensordict[sensor][('Band19')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength19*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band19RSR})
sensordict[sensor][('Band20')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength20*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band20RSR})
sensordict[sensor][('Band21')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength21*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band21RSR})
sensordict[sensor][('Band22')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength22*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band22RSR})
sensordict[sensor][('Band23')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength23*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band23RSR})
sensordict[sensor][('Band24')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength24*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band24RSR})
sensordict[sensor][('Band25')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength25*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band25RSR})
sensordict[sensor][('Band26')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength26*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band26RSR})
sensordict[sensor][('Band27')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength27*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band27RSR})
sensordict[sensor][('Band28')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength28*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band28RSR})
sensordict[sensor][('Band29')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength29*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band29RSR})
sensordict[sensor][('Band30')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength30*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band30RSR})
sensordict[sensor][('Band31')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength31*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band31RSR})
sensordict[sensor][('Band32')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength32*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band32RSR})
sensordict[sensor][('Band33')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength33*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band33RSR})
sensordict[sensor][('Band34')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength34*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band34RSR})
sensordict[sensor][('Band35')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength35*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band35RSR})
sensordict[sensor][('Band36')] = pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength36*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band36RSR})


In [ ]:
# Plot solar spectrum
# http://rredc.nrel.gov/solar/spectra/am1.5/

solar_curve_normal = (solar_curve - solar_curve.min())/(solar_curve.max() - solar_curve.min())
                            
plotly.offline.iplot({
"data": [{"x": sensordict['irradiance'][('solar')].wavelength,\
          "y": sensordict['irradiance'][('solar')].rsr/sensordict['irradiance'][('solar')].rsr.max(),\
          "name": 'Solar Irradiance'},
         #{"x": sensordict['irradiance'][('solar')].wavelength,"y":solar_atmospheric_normal, "name": 'Solar Irradiance'},
         {"x": sensordict['irradiance'][('solar')].wavelength,"y":solar_curve_normal, "name": 'Solar Irradiance'},
        ],
"layout": {"title": "SMARTS Solar Irradiance"}})


In [ ]:
# The holy grail awaits...
sensordict.keys()
#sensordict[sensor][(column)]

In [ ]:
# Example mixing sensordict and individual dict of dataframes access
# We want a function that enables return "sensors that are green" for example

print (plotly.__version__)            # version 1.9.4 required

plotly.offline.iplot({
"data": [{"x": ls8oli[('Green')].Wavelength,"y": ls8oli[('Green')].RSR, "name": 'ls8oli GREEN'},\
         {"x": ls5tm[('Green-L5 TM')].Wavelength,"y": ls5tm[('Green-L5 TM')].RSR, "name": 'ls5tm GREEN'},\
         {"x": ls7etm[('Green-L7')].Wavelength,"y": ls7etm[('Green-L7')].RSR, "name": 'ls7etm GREEN'},\
         #{"x": sensordict['s2amsi'][('Band3')].wavelength,"y": sensordict['s2amsi'][('Band3')].rsr,\
         # "name":'s2amsi Band3'},\
         {"x": sensordict['tmodis'][('Band4')].wavelength,"y": sensordict['tmodis'][('Band4')].rsr,\
          "name":'tmodis Band4'},\
         {"x": sensordict['h8ahi'][('Band 2')].wavelength,"y": sensordict['h8ahi'][('Band 2')].rsr,\
          "name": 'h8ahi Band 2'},\
         {"x": sensordict['aster'][('VNIR_Band1')].wavelength,"y": sensordict['aster'][('VNIR_Band1')].rsr, \
          "name": 'astervnir VNIR1'},\
         {"x": sensordict['qb02'][('Green')].wavelength,"y": sensordict['qb02'][('Green')].rsr, \
          "name": 'quickbird 2 Green'},
         {"x": sensordict['irradiance'][('solar')].wavelength,\
          "y": sensordict['irradiance'][('solar')].rsr/sensordict['irradiance'][('solar')].rsr.max(),\
          "name": 'Solar Irradiance'},
        ],
"layout": {"title": "GREEN Relative Spectral Response"}})


In [ ]:
plotallbands('h8ahi')

In [ ]:
plotallbands('tmodis')

In [ ]:
plotallbands_comparative([('tmodis', 'blue'), ('h8ahi', 'red'), ('viirs', 'green')])

## Synthetic band creation from nominal centre and fwhm

In [ ]:
# Using typical summary sensor info, manufacture the synthetic rsr to compare with measured relative spectral response
ls8oli_summary = pandas.read_csv('l8oli_summary.txt')
eo1_hyperion_summary = pandas.read_csv('Hyperion_cen_fwhm_av.dat', header=1)

In [ ]:
# Synthetic band from centre wavelength and spline roots for fwhm
def synthetic_rsr(samples, bandcentrewavelength,fwhm):
    #returns 1d normalized relative spectral response assuming normal distibution
    sigma = fwhm / 2.35
    normdist = stats.norm.pdf(samples, loc=bandcentrewavelength, scale=sigma)
    response = (normdist - normdist.min())/(normdist.max() - normdist.min())
    
    return(response)

In [ ]:
sensor = 'synth_ls8oli'
sensordict[sensor] = {}
wavelength = np.linspace(1,2500, 2500)

for band, row in ls8oli_summary.iterrows():
    fwhm = ls8oli_summary.FWHM_upper[band] - ls8oli_summary.FWHM_lower[band]
    rsr = synthetic_rsr(wavelength,ls8oli_summary.Central_wavelength[band],fwhm)
    sensordict[sensor][(ls8oli_summary.Band[band])] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})
plotallbands('synth_ls8oli')
plotallbands('ls8oli')

## Hyperspectral example with EO-1 Hyperion

In [ ]:
sensordict['synth_eo1_hyperion'].keys()

In [ ]:
sensor = 'synth_eo1_hyperion'
sensordict[sensor] = {}
wavelength = np.linspace(1,2700, 2700)

for band, row in eo1_hyperion_summary.iterrows():
    rsr = synthetic_rsr(wavelength,eo1_hyperion_summary.cwl[band],eo1_hyperion_summary.fwhm[band])
    sensordict[sensor][(eo1_hyperion_summary.Band[band])] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})
plotallbands('synth_eo1_hyperion')

In [ ]:
def reshape_interpolate(start, stop, samples, npdatatype, input1dwavelength,input1drsr,wlscalefactor):
    wavelength = np.linspace(start,stop,samples, dtype=float)
    rsr = np.nan_to_num(np.interp(wavelength,input1dwavelength*wlscalefactor, input1drsr))
    return wavelength, rsr

## User reference band for matching

In [ ]:
# https://en.wikipedia.org/wiki/Blue 450–495 nm
# https://en.wikipedia.org/wiki/Green 495–570 nm
# https://en.wikipedia.org/wiki/Red 620–740 nm
# https://en.wikipedia.org/wiki/Infrared 750–1400 nm
sensor = 'user_reference'
sensordict[sensor] = {}

wavelength = np.linspace(1,2500, 2500)
rsr = synthetic_rsr(wavelength,478,45)
sensordict[sensor]['BLUE'] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})
rsr = synthetic_rsr(wavelength,533,75)
sensordict[sensor]['GREEN'] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})
rsr = synthetic_rsr(wavelength,680,120)
sensordict[sensor]['RED'] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})
rsr = synthetic_rsr(wavelength,1075,651)
sensordict[sensor]['NIR'] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})

plotallbands('user_reference')

## Compute equivalence metrics for target and reference spectra

In [ ]:
reference = ['tmodis']
#reference = ['user_reference']

target = ['h8ahi','viirs'] #ultimately make this the sensorlist for all potential matches

for sensor in sensordict.keys():
    print(sensor)
    for band in sensordict[sensor]:
        sensordict[sensor][band].wavelength = sensordict[sensor][band].wavelength.astype(int, copy=False)

In [ ]:
# TODO make a def() out of this:
# lists to hold inputs to dataframe once we're done

sensor1list = []
sensor1keys = []
sensor2list = []
sensor2keys = []
pcorrelation = []
emdistance = []
weightedcentredelta = []
areadelta = []
fwhmdelta = []
ks_pvalue= []
ks_statistic = []

# TODO update the interpolation range to fit the min and max wavelength range for the input pairwise comparison
for sensor1 in reference:
    for key1 in sensordict[sensor1].keys():
        for sensor2 in target:
            
            for key2 in sensordict[sensor2].keys():
                sensor1list.append(sensor1)
                sensor1keys.append(key1)
                sensor2list.append(sensor2)
                sensor2keys.append(key2)
                # Find the wavelength range of the rsr values and interpolate within it

                bounds = []
                bounds.append(sensordict[sensor1][(key1)].wavelength\
                              [sensordict[sensor1][(key1)].rsr.replace(0., np.nan).first_valid_index()])
                bounds.append(sensordict[sensor1][(key1)].wavelength\
                              [sensordict[sensor1][(key1)].rsr.replace(0., np.nan).last_valid_index()])
                bounds.append(sensordict[sensor2][(key2)].wavelength\
                              [sensordict[sensor2][(key2)].rsr.replace(0., np.nan).first_valid_index()])
                bounds.append(sensordict[sensor2][(key2)].wavelength\
                              [sensordict[sensor2][(key2)].rsr.replace(0., np.nan).last_valid_index()])

                # Interpolate rsr 
                sensor1wl, sensor1rsr = \
                reshape_interpolate(min(bounds),max(bounds),max(bounds)-min(bounds)+1,'float', sensordict[sensor1][(key1)].wavelength,\
                                    sensordict[sensor1][(key1)].rsr.replace(0., np.nan), 1)
                sensor2wl, sensor2rsr = \
                reshape_interpolate(min(bounds),max(bounds),max(bounds)-min(bounds)+1,'float', sensordict[sensor2][(key2)].wavelength,\
                                    sensordict[sensor2][(key2)].rsr.replace(0., np.nan), 1)

                # A smoothed distrubution seems important for Earth Mover Distance
                A = ndimage.filters.gaussian_filter(sensor1rsr, 10)
                B = ndimage.filters.gaussian_filter(sensor2rsr, 10)

                print ("Calculating equivalence metrics for :", sensor1, key1, "with", sensor2, key2)
                # Earth Mover Distance 
                # From - https://github.com/andreasjansson/python-emd
                try:
                    # normalise - confirm with someone who has maths skills that doing this makes sense - seems to be required for EMD
                    A = (A - A.min())/(A.max() - A.min())
                    B = (B - B.min())/(B.max() - B.min())

                    EMD = emd.emd(range(500), range(500), signal.resample(A,500), signal.resample(B,500))
                except:
                    EMD = 0
                    pass
                emdistance.append(EMD)
                # Pearson correlation coefficient
                pearson = stats.pearsonr(sensor1rsr, sensor2rsr)
                kstest = stats.ks_2samp(sensor1rsr, sensor2rsr)
                ks_pvalue.append(kstest.pvalue)
                ks_statistic.append(kstest.statistic)
                pcorrelation.append(pearson[0])
                # "Area" under each curve
                sensor1trapz = np.trapz(sensor1rsr, sensor1wl)
                sensor2trapz = np.trapz(sensor2rsr, sensor2wl)
                areadelta.append(abs(sensor1trapz - sensor2trapz))
                sensor1mean = np.average(sensor1wl, weights=sensor1rsr)
                sensor2mean = np.average(sensor2wl, weights=sensor2rsr)
                weightedcentredelta.append(abs(sensor1mean - sensor2mean))

                #FWHM as spline roots
                spline1 = UnivariateSpline(sensor1wl, A-A.max()/2, s=0)
                spline2 = UnivariateSpline(sensor2wl, B-B.max()/2, s=0)
                try:
                    sensor1r1, sensor1r2 = spline1.roots()
                    sensor2r1, sensor2r2 = spline2.roots()
                except:
                    sensor1r1 = 100.
                    sensor1r2 = 100. 
                    sensor2r1 = 100.
                    sensor2r2 = 100. 
                    pass
                fwhmdelta.append(abs((sensor1r2-sensor1r1)-(sensor2r2-sensor2r1)))

                # Reduce the number of plots output, use a correlation threshold to determine whether to display

                if (pearson[0] > 0.6):
                    plotly.offline.iplot({
                        "data": [{"x": sensor1wl,"y": sensor1rsr, "name": sensor1+"-"+key1, "line": dict(color = ('rgb(255, 1, 1)'))},\
                                 {"x": pandas.Series([sensor1mean, sensor1mean]),"y": pandas.Series([0,1]), "name": 'mean wavelength',\
                                  "line": dict(color = ('rgb(255, 1, 1)'), width = 1, dash = 'dash')},\
                                 {"x": sensor1wl,"y": A, "name": 'A',\
                                  "name": 'gaussian', "line": dict(color = ('rgb(255, 1, 1)'), width = 1, dash = 'dot')},\
                                 {"x": pandas.Series([sensor1r1, sensor1r1]),"y": pandas.Series([0,1]), "name": 'fwhm root1',\
                                  "line": dict(color = ('rgb(255, 1, 1)'), width = 1, dash = 'dashdot')},
                                 {"x": pandas.Series([sensor1r2, sensor1r2]),"y": pandas.Series([0,1]), "name": 'fwhm root2',\
                                  "line": dict(color = ('rgb(255, 1, 1)'), width = 1, dash = 'dashdot')},


                                 {"x": sensor2wl,"y": sensor2rsr, "name": sensor2+"-"+key2, "line": dict(color = ('rgb(1, 1, 255)'))},\
                                 {"x": pandas.Series([sensor2mean, sensor2mean]),"y": pandas.Series([0,1]) , "name": 'mean wavelength',\
                                  "line": dict(color = ('rgb(1, 1, 255)'), width = 1, dash = 'dash')},
                                 {"x": sensor2wl,"y": B, "name": 'B',\
                                  "name": 'gaussian', "line": dict(color = ('rgb(1, 1, 255)'), width = 1, dash = 'dot')},\
                                 {"x": pandas.Series([sensor2r1, sensor2r1]),"y": pandas.Series([0,1]), "name": 'fwhm root1',\
                                  "line": dict(color = ('rgb(1, 1, 255)'), width = 1, dash = 'dashdot')},
                                 {"x": pandas.Series([sensor2r2, sensor2r2]),"y": pandas.Series([0,1]), "name": 'fwhm root2',\
                                  "line": dict(color = ('rgb(1, 1, 255)'), width = 1, dash = 'dashdot')},\

                                ],

                        "layout": {"title": 'Equivalence match: '+sensor1+' '+key1+' and '+sensor2+' '+key2}})

                del bounds, EMD, sensor1trapz, sensor2trapz, pearson, A, B, sensor1wl, sensor1rsr,sensor2wl, sensor2rsr,\
                sensor1r1, sensor1r2, sensor2r1, sensor2r2

    spectrumcomparison = pandas.DataFrame({'sensor1' : sensor1list, 'sensor1keys' : sensor1keys, 'sensor2': sensor2list,\
                                           'sensor2keys': sensor2keys, 'pcorrelation' : pcorrelation, 'ks_pvalue': ks_pvalue,\
                                           'ks_statistic': ks_statistic, 'distance': emdistance,\
                                           'areadelta': areadelta, 'weightedcentredelta': weightedcentredelta, 'fwhmdelta': fwhmdelta})


## Summarize the equivalence metrics

In [ ]:
spectrumcomparison.sort_values('sensor1keys', ascending=True)

## Set the matching threshold here for the equivalence metrics

In [ ]:
match_limits = {'pcorrelation': 0.3, 'distance': 50, 'weightedcentredelta': 70, 'areadelta': 100, 'fwhmdelta': 50}
spectrumcomparison[(spectrumcomparison.pcorrelation > match_limits['pcorrelation']) &\
                   (spectrumcomparison.distance < match_limits['distance']) &\
                   (spectrumcomparison.weightedcentredelta < match_limits['weightedcentredelta']) &\
                   (spectrumcomparison.areadelta < match_limits['areadelta']) &\
                   (spectrumcomparison.areadelta < match_limits['fwhmdelta'])
                   ]

In [ ]:
# Write this puppy out as per Josh's
hdf = pandas.HDFStore('store.h5')
#TODO ... later

In [ ]:
# http://www.eoc.csiro.au/hswww/oz_pi/docs/fwhm_notes.pdf From David Jupp
# P.M.  Teillet,  G.  Fedosejevs,  K.J.  Thome,  and  J.L.  Barker,  “Impacts  of  spectral  band  difference  effects  on  radiometric  
#cross-calibration between satellite sensors in the solar-reflective spectral domain,” 
#Remote Sensing of Environment
#, 110, pp. 
#393–409, 2007
# http://stackoverflow.com/questions/10623448/making-gaussians-constrained-by-the-fwhm

In [ ]:
# Solar irradiance curve - how much radiant energy is hitting the sensor...proportionally? 
# http://rredc.nrel.gov/solar/spectra/am1.5/
# TODO - done above but difficult to track down the normalised transmission data for a representative atmosphere
# across the 300 to 15000 nanometer spectrum

In [ ]:
# TODO Band resampling to provide proportional inclusion of multiple summed bands in synthetic band (i.e. with pan band,
# or hyperspectral, narrow bands to broad band)

In [ ]:
# Spectral library stuff for much later - http://speclab.cr.usgs.gov/ 